<a href="https://colab.research.google.com/github/kellianneyang/project-exploration/blob/main/csv_merging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grades Project: Merge .csv files into one dataframe

# Goal

In this notebook, the goal is to merge all .csv files from the Kaggle dataset into one .csv file to use for the project.

The main steps will be:
- importing each .csv file
- renaming columns as necessary (some columns have the same names in separate .csv files and need to be renamed so they are distinct)
- making all data lowercase (to eliminate non-matching data if caps are different)
- merging all data together
  - this may include casting columns to a different datatype to merge on that column; if this needs to happen, some rows may be dropped after checking that there is no data in the other .csvs that is associated with the dropped rows first

This notebook will NOT:
- delete any columns
- create any new columns
- fill any missing values
- check for any duplicates

## Overview


To use the data provided in this dataset (https://www.kaggle.com/datasets/Madgrades/uw-madison-courses), which is provided in multiple .csv files, for a machine learning model, we will need to merge the files to create one .csv file.

## .csv files and columns

Included in dataset as separate .csv files:
- schedules.csv: each row is a unique potential schedule
    - (schedule) uuid: unique identifier of schedules
    - start_time: start of class, in minutes
    - end_time: end of class, in minutes (drop -- high co-linearity with start_time)
    - mon: boolean, if class meets on monday
    - tues: "
    - wed: "
    - thurs: "
    - fri: "
    - sat: "
    - sun: "
      - sat and sun have very few values; combine to get more observations
- subjects.csv: each row is a unique subject
    - (subject) code: 3-digit unique identifier of subjects
    - (subject) name: name of subject
    - (subject) abbreviation: abbreviation (e.g. ENGL for English)
- teachings.csv: each row is a unique instructor
    - instructor_id: numeric unique identifier of instructor
    - section_uuid: section taught by instructor
- subject_memberships.csv: each row is a course offering (course offered in certain term; does not encompass all sections in that term)
    - subject_code: subject code associated with course offering
    - course_offering_uuid: unique identifier of course offerings
- sections.csv: each row is a section (specific instance of course at certain time in certain place in certain term)
    - (section) uuid: unique identifier of section (alphanumeric)
      - could have multiple section uuids for cross-listed sections
    - course_offering_uuid: unique identifier of course offering (course offered in certain term but encompasses all sections in that term) 
    - section_type: 3-letter identifier (e.g., LEC for lecture)
    - (section) number: 1-3-number of section (e.g., 301 for section 301)
    - room_uuid: unique identifier of room and building where section is held (including online and off-campus designations)
    - schedule_uuid: unique identifier of schedule for section
- rooms.csv: each row is a specific place
    - (room) uuid: unique identifier of building and room (inclduing off-campus and online)
    - facility_code: unique identifier of building
    - room_code: number of room within a building
- instructors.csv (will not use -- is essentially a duplicate of teachings; can use to look up instructors' names): each row is an instructor
    - (instructor) id: unique identifier of instructor
    - (instructor) name: instructor's name
- grade_distributions.csv: each row is a section's grades
    - course_offering_uuid: identifies the course offering (certain course in certain term, but not broken down into sections)
    - section_number: number of section for grades
    - a_count: number of As
    - ab_count: number of ABs (can be combined with As as needed)
    - b_count: number of Bs
    - bc_count: number of BCs (can be combined with Bs as needed)
    - c_count: number of Cs
    - d_count: number of Ds
    - f_count: number of Fs
    - s_count: satisfactory
    - u_count: unstatisfactory
    - cr_count: credit
    - n_count: no credit
    - p_count
    - i_count: incomplete
    - nw_count: no work
    - nr_count
- course_offerings.csv: each row is a course offering (course offered in certain term)
    - (course offering) uuid: unique identifier of course offering
    - course_uuid: course that the course offering belongs to
    - term_code: academic term when course offering was held
    - (course offering) name: name associate with course offering (can be different than course name)
- courses.csv: each row is a course (abstract; not associated with specific course offering)
    - (course) uuid: uniquely identifies the course
    - (course) name: name of course
    - (course) number: number in course catalog (e.g. 101 for ENGL 101)

# Preliminary steps

In [ ]:
# import libraries
import pandas as pd

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# sections.csv

Sections.csv contains the column 'section_uuid' that has a correlate column in teachings.csv. In teachings.csv, the only other column is 'teacher_id'. 

In this section, we merge 'teacher_id' into the sections.csv data. We will keep 'section_uuid' in the data for now, but it may prove to be a high-cardinality feature, with almost as many values as observations. So, we will keep it in mind for later, when we can experiment to see if dropping it will help our machine learning algorithm make better predictions.

## Import

In [ ]:
# create df from .csv
path = '/content/drive/MyDrive/Coding Dojo/Project 2/UW Madison Courses and Grades 2006-2017/sections.csv'
sections_df = pd.read_csv(path)

## Inspect

In [ ]:
# inspect df
sections_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315602 entries, 0 to 315601
Data columns (total 6 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   uuid                  315602 non-null  object
 1   course_offering_uuid  315602 non-null  object
 2   section_type          315602 non-null  object
 3   number                315602 non-null  int64 
 4   room_uuid             209644 non-null  object
 5   schedule_uuid         315602 non-null  object
dtypes: int64(1), object(5)
memory usage: 14.4+ MB


## Rename columns & make lowercase

In [ ]:
# rename 'number' to 'section_number' to avoid confusion with future 'number' 
# columns in other .csv files
sections_df.rename(columns = {'number': 'section_number'}, inplace = True)

In [ ]:
# rename 'uuid' to 'section_uuid' to avoid confusion with future 'uuid'
# columns in other .csv files
sections_df.rename(columns = {'uuid': 'section_uuid'}, inplace = True)

In [ ]:
# check
sections_df.columns

Index(['section_uuid', 'course_offering_uuid', 'section_type',
       'section_number', 'room_uuid', 'schedule_uuid'],
      dtype='object')

In [ ]:
# make all values lowercase to standardize
sections_df = sections_df.applymap(lambda x: x.lower() if type(x) == str else x)

# teachings.csv

## Import

In [ ]:
# create df from teachings.csv
path = '/content/drive/MyDrive/Coding Dojo/Project 2/UW Madison Courses and Grades 2006-2017/teachings.csv'
teachings_df = pd.read_csv(path)

## Inspect

In [ ]:
# inspect new data
teachings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315211 entries, 0 to 315210
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   instructor_id  315211 non-null  int64 
 1   section_uuid   315211 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.8+ MB


## Rename columns & make lowercase

In [ ]:
# make all values lowercase to standardize
teachings_df = teachings_df.applymap(lambda x: x.lower() if type(x) == str else x)

## Merge and fix

### Merge

In [ ]:
# merge sections_df and teachings_df on 'section_uuid'
# into new df 'big_df'
big_df = pd.merge(sections_df, teachings_df, how = 'outer', on = 'section_uuid')

In [ ]:
# inspect
big_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 315602 entries, 0 to 315601
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   section_uuid          315602 non-null  object 
 1   course_offering_uuid  315602 non-null  object 
 2   section_type          315602 non-null  object 
 3   section_number        315602 non-null  int64  
 4   room_uuid             209644 non-null  object 
 5   schedule_uuid         315602 non-null  object 
 6   instructor_id         315211 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 19.3+ MB


# rooms.csv

## Import

In [ ]:
path = '/content/drive/MyDrive/Coding Dojo/Project 2/UW Madison Courses and Grades 2006-2017/rooms.csv'
rooms_df = pd.read_csv(path)

## Inspect

In [ ]:
# inspect
rooms_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   uuid           1350 non-null   object
 1   facility_code  1350 non-null   object
 2   room_code      1348 non-null   object
dtypes: object(3)
memory usage: 31.8+ KB


## Rename columns & make lowercase

In [ ]:
# rename uuid to room_uuid
rooms_df.rename(columns = {'uuid': 'room_uuid'}, inplace = True)

In [ ]:
# make all values lowercase to standardize
rooms_df = rooms_df.applymap(lambda x: x.lower() if type(x) == str else x)

## Merge and fix

### Merge

In [ ]:
# merge with big_df on 'room_uuid'
big_df = pd.merge(big_df, rooms_df, how = 'outer', on = 'room_uuid')

In [ ]:
# check
big_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 315602 entries, 0 to 315601
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   section_uuid          315602 non-null  object 
 1   course_offering_uuid  315602 non-null  object 
 2   section_type          315602 non-null  object 
 3   section_number        315602 non-null  int64  
 4   room_uuid             209644 non-null  object 
 5   schedule_uuid         315602 non-null  object 
 6   instructor_id         315211 non-null  float64
 7   facility_code         209644 non-null  object 
 8   room_code             204052 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 24.1+ MB


# schedules.csv

## Import

In [ ]:
path = '/content/drive/MyDrive/Coding Dojo/Project 2/UW Madison Courses and Grades 2006-2017/schedules.csv'
schedules_df = pd.read_csv(path)

## Inspect

In [ ]:
# inspect
schedules_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4467 entries, 0 to 4466
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   uuid        4467 non-null   object
 1   start_time  4467 non-null   int64 
 2   end_time    4467 non-null   int64 
 3   mon         4467 non-null   bool  
 4   tues        4467 non-null   bool  
 5   wed         4467 non-null   bool  
 6   thurs       4467 non-null   bool  
 7   fri         4467 non-null   bool  
 8   sat         4467 non-null   bool  
 9   sun         4467 non-null   bool  
dtypes: bool(7), int64(2), object(1)
memory usage: 135.4+ KB


## Rename columns & make lowercase

In [ ]:
# replace 'uuid' with 'schedule_uuid' to be able to merge
schedules_df.rename(columns = {'uuid': 'schedule_uuid'}, inplace = True)

In [ ]:
# make all values lowercase to standardize
schedules_df = schedules_df.applymap(lambda x: x.lower() if type(x) == str else x)

## Merge and fix

### Merge

In [ ]:
# merge schedules_df with big_df on 'schedule_uuid'
big_df = pd.merge(big_df, schedules_df, how = 'outer', on = 'schedule_uuid')

In [ ]:
# check
big_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 315602 entries, 0 to 315601
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   section_uuid          315602 non-null  object 
 1   course_offering_uuid  315602 non-null  object 
 2   section_type          315602 non-null  object 
 3   section_number        315602 non-null  int64  
 4   room_uuid             209644 non-null  object 
 5   schedule_uuid         315602 non-null  object 
 6   instructor_id         315211 non-null  float64
 7   facility_code         209644 non-null  object 
 8   room_code             204052 non-null  object 
 9   start_time            315602 non-null  int64  
 10  end_time              315602 non-null  int64  
 11  mon                   315602 non-null  bool   
 12  tues                  315602 non-null  bool   
 13  wed                   315602 non-null  bool   
 14  thurs                 315602 non-null  bool   
 15  

# subject_memberships.csv and subjects.csv 

## subject_memberships.csv

### Import

In [ ]:
path = '/content/drive/MyDrive/Coding Dojo/Project 2/UW Madison Courses and Grades 2006-2017/subject_memberships.csv'
subject_memberships_df = pd.read_csv(path)

### Inspect

In [ ]:
# inspect
subject_memberships_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95314 entries, 0 to 95313
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   subject_code          95314 non-null  int64 
 1   course_offering_uuid  95314 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.5+ MB


### Rename columns & make lowercase

In [ ]:
# make all values lowercase to standardize
subject_memberships_df = subject_memberships_df.applymap(lambda x: x.lower() if\
                                                         type(x) == str else x)

## subjects.csv

### Import

In [ ]:
# load subjects.csv
path = '/content/drive/MyDrive/Coding Dojo/Project 2/UW Madison Courses and Grades 2006-2017/subjects.csv'
subjects_df = pd.read_csv(path)

### Inspect

In [ ]:
# check out subjects_df
subjects_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   code          200 non-null    object
 1   name          200 non-null    object
 2   abbreviation  200 non-null    object
dtypes: object(3)
memory usage: 4.8+ KB


### Rename columns & make lowercase

In [ ]:
subjects_df.rename(columns = {'code': 'subject_code', 
                              'name': 'subject_name',
                              'abbreviation': 'subject_abbreviation'},
                   inplace = True)

In [ ]:
# make all values lowercase to standardize
subjects_df = subjects_df.applymap(lambda x: x.lower() if type(x) == str else x)

## Merge subjects.csv and subject_memberships.csv

### Merge

In [ ]:
# cast 'subject_code' as int so can merge with df
# subjects_df['subject_code'] = subjects_df['subject_code'].astype(int) 
# throws error for value 'zzz'

In [ ]:
# look up 'ZZZ' in subject_code
subjects_df.loc[subjects_df['subject_code'] == 'zzz', ['subject_code']] 
# index 127

,subject_code
127,zzz


In [ ]:
subjects_df.loc[127, :] 
# 'zzz' should be assigned to the subject_code for registrar-administrative use

subject_code                                     zzz
subject_name            registrar-administrative use
subject_abbreviation                        adminuse
Name: 127, dtype: object

In [ ]:
subject_memberships_df.loc[subject_memberships_df['subject_code'] == 'zzz', :]
# there are no courses in the dataset with this subject_code, so we can delete

,subject_code,course_offering_uuid


In [ ]:
subjects_df.drop(127, axis = 0, inplace = True)

In [ ]:
# check
subjects_df.loc[subjects_df['subject_name'] == 'registrar-administrative use', :]

,subject_code,subject_name,subject_abbreviation


In [ ]:
# cast 'subject_code' as int so can merge with df
# subjects_df['subject_code'] = subjects_df['subject_code'].astype(int) 
# throws error for value 'sab'

In [ ]:
# look up 'sab' in subject_code
subjects_df.loc[subjects_df['subject_code'] == 'sab', ['subject_code']] 
# index 173

,subject_code
173,sab


In [ ]:
subjects_df.loc[173, :] 
# 'sab' should be assigned to the subject_code for 'international academic programs - study abroad'

subject_code                                                       sab
subject_name            international academic programs - study abroad
subject_abbreviation                                          stdyabrd
Name: 173, dtype: object

In [ ]:
subject_memberships_df.loc[subject_memberships_df['subject_code'] == 'sab', :]
# there are no courses in the dataset with this subject_code, so we can delete

,subject_code,course_offering_uuid


In [ ]:
subjects_df.drop(173, axis = 0, inplace = True)

In [ ]:
# check
subjects_df.loc[subjects_df['subject_name'] == 'international academic programs - study abroad', :]

,subject_code,subject_name,subject_abbreviation


In [ ]:
# cast 'subject_code' as int so can merge with df
subjects_df['subject_code'] = subjects_df['subject_code'].astype(int)

In [ ]:
# merge on 'subject_code
complete_subjects_df = pd.merge(subject_memberships_df, 
                              subjects_df, 
                              how = 'outer', 
                              on = 'subject_code')

In [ ]:
# inspect
complete_subjects_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95319 entries, 0 to 95318
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   subject_code          95319 non-null  int64 
 1   course_offering_uuid  95314 non-null  object
 2   subject_name          95319 non-null  object
 3   subject_abbreviation  95319 non-null  object
dtypes: int64(1), object(3)
memory usage: 3.6+ MB


## Merge with big_df

### Merge

In [ ]:
# merge complete_subjects_df with big_df on 'course_offering_uuid'
big_df = pd.merge(big_df, 
                  complete_subjects_df, 
                  how = 'outer', 
                  on = 'course_offering_uuid')

In [ ]:
# check
big_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 365133 entries, 0 to 365132
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   section_uuid          365128 non-null  object 
 1   course_offering_uuid  365128 non-null  object 
 2   section_type          365128 non-null  object 
 3   section_number        365128 non-null  float64
 4   room_uuid             255639 non-null  object 
 5   schedule_uuid         365128 non-null  object 
 6   instructor_id         364408 non-null  float64
 7   facility_code         255639 non-null  object 
 8   room_code             249390 non-null  object 
 9   start_time            365128 non-null  float64
 10  end_time              365128 non-null  float64
 11  mon                   365128 non-null  object 
 12  tues                  365128 non-null  object 
 13  wed                   365128 non-null  object 
 14  thurs                 365128 non-null  object 
 15  

# course_offerings.csv

## Import

In [ ]:
path = '/content/drive/MyDrive/Coding Dojo/Project 2/UW Madison Courses and Grades 2006-2017/course_offerings.csv'
course_offerings_df = pd.read_csv(path)

## Inspect

In [ ]:
# inspect
course_offerings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81452 entries, 0 to 81451
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   uuid         81452 non-null  object
 1   course_uuid  81452 non-null  object
 2   term_code    81452 non-null  int64 
 3   name         78916 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.5+ MB


## Rename columns & make lowercase

In [ ]:
# rename 'name' to 'course_offering_name' to avoid confusion with 'course_name' 
# (official course name and particular-term course name can differ)

# rename 'uuid' to 'course_offering_uuid' to avoid confusion with 'course_uuid'
course_offerings_df.rename(columns = {'name': 'course_offering_name', 
                                      'uuid': 'course_offering_uuid'}, 
                           inplace = True)

In [ ]:
# inspect
course_offerings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81452 entries, 0 to 81451
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   course_offering_uuid  81452 non-null  object
 1   course_uuid           81452 non-null  object
 2   term_code             81452 non-null  int64 
 3   course_offering_name  78916 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.5+ MB


In [ ]:
# make all values lowercase to standardize
course_offerings_df = course_offerings_df.applymap(lambda x: x.lower() if \
                                                   type(x) == str else x)

## Merge

### Merge

In [ ]:
# merge with big_df on course_offering_uuid
big_df = pd.merge(big_df, 
                  course_offerings_df, 
                  how = 'outer', 
                  on = 'course_offering_uuid')

In [ ]:
# check
big_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 365133 entries, 0 to 365132
Data columns (total 24 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   section_uuid          365128 non-null  object 
 1   course_offering_uuid  365128 non-null  object 
 2   section_type          365128 non-null  object 
 3   section_number        365128 non-null  float64
 4   room_uuid             255639 non-null  object 
 5   schedule_uuid         365128 non-null  object 
 6   instructor_id         364408 non-null  float64
 7   facility_code         255639 non-null  object 
 8   room_code             249390 non-null  object 
 9   start_time            365128 non-null  float64
 10  end_time              365128 non-null  float64
 11  mon                   365128 non-null  object 
 12  tues                  365128 non-null  object 
 13  wed                   365128 non-null  object 
 14  thurs                 365128 non-null  object 
 15  

# courses.csv

## Import

In [ ]:
path = '/content/drive/MyDrive/Coding Dojo/Project 2/UW Madison Courses and Grades 2006-2017/courses.csv'
courses_df = pd.read_csv(path)

## Inspect

In [ ]:
# inspect
courses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9306 entries, 0 to 9305
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   uuid    9306 non-null   object
 1   name    9264 non-null   object
 2   number  9306 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 218.2+ KB


## Rename columns & make lowercase

In [ ]:
# rename 'uuid' to 'course_uuid', 'name' to 'course_name', 
# 'number' to 'course_number' to avoid confusion
courses_df.rename(columns = {'uuid': 'course_uuid', 
                             'name': 'course_name', 
                             'number': 'course_number'}, 
                  inplace = True)

In [ ]:
# check
courses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9306 entries, 0 to 9305
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   course_uuid    9306 non-null   object
 1   course_name    9264 non-null   object
 2   course_number  9306 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 218.2+ KB


In [ ]:
# make all values lowercase to standardize
courses_df = courses_df.applymap(lambda x: x.lower() if type(x) == str else x)

## Merge

### Merge

In [ ]:
# merge with big_df on course_uuid
big_df = pd.merge(big_df, 
                  courses_df, 
                  how = 'outer', 
                  on = 'course_uuid')

In [ ]:
# inspect
big_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 365133 entries, 0 to 365132
Data columns (total 26 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   section_uuid          365128 non-null  object 
 1   course_offering_uuid  365128 non-null  object 
 2   section_type          365128 non-null  object 
 3   section_number        365128 non-null  float64
 4   room_uuid             255639 non-null  object 
 5   schedule_uuid         365128 non-null  object 
 6   instructor_id         364408 non-null  float64
 7   facility_code         255639 non-null  object 
 8   room_code             249390 non-null  object 
 9   start_time            365128 non-null  float64
 10  end_time              365128 non-null  float64
 11  mon                   365128 non-null  object 
 12  tues                  365128 non-null  object 
 13  wed                   365128 non-null  object 
 14  thurs                 365128 non-null  object 
 15  

# grade_distributions.csv

## Import

In [ ]:
path = '/content/drive/MyDrive/Coding Dojo/Project 2/UW Madison Courses and Grades 2006-2017/grade_distributions.csv'
grades_df = pd.read_csv(path)

## Inspect

In [ ]:
grades_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193262 entries, 0 to 193261
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   course_offering_uuid  193262 non-null  object
 1   section_number        193262 non-null  int64 
 2   a_count               193262 non-null  int64 
 3   ab_count              193262 non-null  int64 
 4   b_count               193262 non-null  int64 
 5   bc_count              193262 non-null  int64 
 6   c_count               193262 non-null  int64 
 7   d_count               193262 non-null  int64 
 8   f_count               193262 non-null  int64 
 9   s_count               193262 non-null  int64 
 10  u_count               193262 non-null  int64 
 11  cr_count              193262 non-null  int64 
 12  n_count               193262 non-null  int64 
 13  p_count               193262 non-null  int64 
 14  i_count               193262 non-null  int64 
 15  nw_count         

## Rename columns & make lowercase

In [ ]:
# make all values lowercase to standardize
grades_df = grades_df.applymap(lambda x: x.lower() if type(x) == str else x)

In [ ]:
# rename 'section_number' to 'grades_section_number' (there is already a 
# 'section_number' column in sections_df)
grades_df.rename(columns = {'section_number': 'grades_section_number'},
                  inplace = True)

## Merge: Option 1 (on single column 'course_offering_uuid')

In [ ]:
# merge with big_df on course_offering_uuid
big_df1 = pd.merge(big_df, 
                  grades_df, 
                  how = 'outer', 
                  on = 'course_offering_uuid')

In [ ]:
# inspect
big_df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2211665 entries, 0 to 2211664
Data columns (total 43 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   section_uuid           object 
 1   course_offering_uuid   object 
 2   section_type           object 
 3   section_number         float64
 4   room_uuid              object 
 5   schedule_uuid          object 
 6   instructor_id          float64
 7   facility_code          object 
 8   room_code              object 
 9   start_time             float64
 10  end_time               float64
 11  mon                    object 
 12  tues                   object 
 13  wed                    object 
 14  thurs                  object 
 15  fri                    object 
 16  sat                    object 
 17  sun                    object 
 18  subject_code           int64  
 19  subject_name           object 
 20  subject_abbreviation   object 
 21  course_uuid            object 
 22  term_code         

## Merge: Option 2 (on two columns: 'course_offering_uuid' and 'section_number')

In [ ]:
grades_df2 = grades_df.copy()

In [ ]:
# in grades_df2 the grades_section_number column is int, but in big_df2 the section_number column is a float,
# so when they are converted to strings they will be different
# solution: cast grades_df2['grades_section_number'].astype(float)

grades_df2['grades_section_number'] = grades_df2['grades_section_number'].astype(float)

In [ ]:
# make a column in grades_df2 that combines 'course_offering_uuid' and 'grades_section_number'
# to use as the merge column
grades_df2['course_and_section'] = grades_df2['course_offering_uuid'].astype(str) + "_" + grades_df2['grades_section_number'].astype(str)

In [ ]:
# check
grades_df2['course_and_section'].sample(10)

46749      9e9044fc-e680-35c1-9841-8054ac2b5231_2.0
92688     df0b85ba-1abd-354c-a909-3a6760606931_66.0
174583     936320ac-40e8-3aa1-8395-ecfcb5249152_1.0
64216     4d0899be-73fa-38ca-83ff-13386bb3c4fa_11.0
33307      93bdc34c-8da1-3ed3-83ba-bb33e6819bb2_1.0
100542    9e9105e8-5333-3c0b-be2d-13bd7a8ddd78_50.0
173353     2962cb6b-311f-32bb-b04d-bed65715775c_1.0
114906    205cc645-d7be-34a2-a498-613317cb9532_43.0
33517      9f5be4ff-4896-3a14-b8e9-1d0ab16ac05f_1.0
175311     98a05e01-8049-361d-807f-a8e73c719c36_1.0
Name: course_and_section, dtype: object

In [ ]:
# drop columns 'course_offering_uuid' and 'grades_section_number'
grades_df2.drop(columns = ['course_offering_uuid', 'grades_section_number'], inplace = True)

In [ ]:
# see if new column 'course_and_section' uniquely identifies all rows in grades_df2
print(f"There are {grades_df2['course_and_section'].nunique()} unique values in \
the 'course_and_section' column in grades_df2 and there are {len(grades_df2)} \
rows in grades_df2.")

# yes, this column uniquely identifies all rows in grades_df2

There are 193262 unique values in the 'course_and_section' column in grades_df2 and there are 193262 rows in grades_df2.


In [ ]:
big_df2 = big_df.copy()

In [ ]:
# make a column in big_df2 same as above for grades_df2
big_df2['course_and_section'] = big_df2['course_offering_uuid'].astype(str) + "_" + big_df2['section_number'].astype(str)

In [ ]:
# check
big_df2['course_and_section'].sample(10)

224611    516f299c-0156-340a-8e00-a992ec795346_304.0
37034      ed61de03-eba8-3787-b4f8-77ac41e26f57_21.0
80081      c02d642b-1352-3ccb-aa52-f92ae7b8803d_42.0
147723    0b67af02-a4d8-3d4f-8c90-1f638959017a_330.0
169030      2a54c6d3-fb2d-3fe8-b3f9-e86856d59e98_1.0
308019      87e5ab0a-6c95-3951-8536-83530c86d3aa_1.0
195477    46c8a4ab-ca5a-3e17-8cfb-87995d8bafb0_615.0
196317    e64fcae8-f738-3bd8-9d03-c96cd7d285f3_610.0
70318       dbb9cbac-0f63-31a1-8fc9-57eea6611f1f_1.0
219032    d5d0eaa5-92af-3ac9-99a1-b6b16c3c5a3f_305.0
Name: course_and_section, dtype: object

In [ ]:
# see if new column 'course_and_section' uniquely identifies all rows in big_df2
print(f"There are {big_df2['course_and_section'].nunique()} unique values in \
the 'course_and_section' column in big_df2 and there are {len(big_df2)} \
rows in big_df2.")

# no, this column does not uniquely identify rows in big_df2
# so, before we merge on this column, we need to figure out how the rows differ
# in big_df2, so that we don't merge the same grade distribution to more than
# one row in big_df2

There are 277895 unique values in the 'course_and_section' column in big_df2 and there are 365133 rows in big_df2.


In [ ]:
# see which rows share the same value in the 'course_and_section' column in big_df2, and how many rows share the same value
big_df2['course_and_section'].value_counts()

# one 'course_and_section' includes 50 rows, and there are more 'course_and_section' values that share multiple rows

e6fe4acb-9375-3b46-9d16-6d6ae716d80a_1.0     50
76b9c458-d3c2-38c4-951f-69b6900bd7fe_1.0     45
febd8499-f935-3941-891f-79e2e34fa74e_1.0     42
7b8aa2b9-560f-34a6-aa4b-153d1abfb542_1.0     40
ea3b717c-d66b-30dc-8b37-964d9688295f_1.0     40
                                             ..
24abdcdb-d322-32f8-872b-e877e9d8d68e_1.0      1
24abdcdb-d322-32f8-872b-e877e9d8d68e_10.0     1
24abdcdb-d322-32f8-872b-e877e9d8d68e_18.0     1
24abdcdb-d322-32f8-872b-e877e9d8d68e_13.0     1
ad38492a-f913-3c3b-84bf-84b7f93cc925_1.0      1
Name: course_and_section, Length: 277895, dtype: int64

In [ ]:
# investigate differences among rows that share the same value in the 'course_and_section' column in big_df2


# look at the 'course_and_section' value that is copied in 50 rows, and see what differs among those 50 rows
section1_df = big_df2[big_df2['course_and_section'] == 'e6fe4acb-9375-3b46-9d16-6d6ae716d80a_1.0']

for col in section1_df.columns:
  print(f"column: {col}")
  print(section1_df[col].value_counts(dropna = False))
  print()

# they differ on 'section_uuid', 'instructor_id', 'subject_name', 'facility_code'

column: section_uuid
6fedc217-81a8-3bc4-9d57-5e10481282dc    10
b0e86f13-692f-3f7d-8572-c94a8e7d1f07    10
9465796d-7f08-3569-ac22-92b20cd245a1    10
86a1c582-983c-3dbf-bc37-0cbd28407651    10
d7ba74dc-2da5-39ba-845d-0422fb1909ec    10
Name: section_uuid, dtype: int64

column: course_offering_uuid
e6fe4acb-9375-3b46-9d16-6d6ae716d80a    50
Name: course_offering_uuid, dtype: int64

column: section_type
fld    50
Name: section_type, dtype: int64

column: section_number
1.0    50
Name: section_number, dtype: int64

column: room_uuid
04368a56-c959-3e4b-8b3d-f4cc3538fea5    30
NaN                                     20
Name: room_uuid, dtype: int64

column: schedule_uuid
f2d66a4d-0c08-3b48-abf6-649fffd7ae90    50
Name: schedule_uuid, dtype: int64

column: instructor_id
761703.0     20
788586.0     20
3677061.0    10
Name: instructor_id, dtype: int64

column: facility_code
off campus    30
NaN           20
Name: facility_code, dtype: int64

column: room_code
NaN    50
Name: room_code, dtype:

In [ ]:
# to compress these 50 rows into one, we could:
# assign 'multiple' for 'facility_code', 'instructor_id', and 'section_uuid' ('section_uuid' will eventually be dropped anyway)
# assign 'engineering' for 'subject_name'
# manually go through all other values for 'course_and_section' with multiple rows and re-assign other column values

# the above would be a good strategy to make sure all data is preserved, but would be difficult/time-consuming to implement
# so for the sake of simplicity (and meeting the due date deadline),
# for now, we will drop every row that shares a 'course_and_section' column value with any other row
# but before we do that, let's see how much of our data that strategy would delete!

# get the value_counts again for the 'course_and_section' column, save in an array
course_and_section_value_counts_array = big_df2['course_and_section'].value_counts()

In [ ]:
# get a list of the 'course_and_section' values that have multiple rows

# create empty list
duplicated_course_and_section_list = []

# creat empty counter to count number of rows that are duplicated on 'course_and_section'
num_duplicated_rows = 0

# loop through each 'course_and_section' value and its 'value_count' in the array
for course_and_section, value_count in course_and_section_value_counts_array.items():

  # if the 'course_and_section' string represents is duplicated (appears in more than one row)
  if value_count > 1:

    # add the 'course_and_section' string to the duplicated_course_and_section_list
    duplicated_course_and_section_list.append(course_and_section)

    # add the value_count to the num_duplicated_rows
    num_duplicated_rows += value_count

print(len(duplicated_course_and_section_list)) # how many rows are duplicated on 'course_and_section'
print(num_duplicated_rows) # in big_df2

48793
136031


In [ ]:
# what percent of our data does this represent?
# number of duplicated rows in big_df2 over number of rows in big_df2
print((num_duplicated_rows / len(big_df2)) * 100)

# we are losing all the "duplicated" rows (any rows that share their 'course_and_section' value 
# with another row) in big_df2 but we are also losing 48,793 grade distributions (assuming there
# is one grade distribution (in grades_df2) per 'course_and_section' value in big_df2)
((len(duplicated_course_and_section_list) / len(grades_df2)) * 100)

# i think this dataset is big enough that this loss of data is okay for now,
# but if i had more time i would go back and implement the more difficult strategy
# of compressing the duplicated rows into one so we wouldn't lose the grade data
# for that row

37.25519194375748


25.247073920377517

In [ ]:
# check duplicated course and section list
for item in duplicated_course_and_section_list[:20]:
  print(item)

e6fe4acb-9375-3b46-9d16-6d6ae716d80a_1.0
76b9c458-d3c2-38c4-951f-69b6900bd7fe_1.0
febd8499-f935-3941-891f-79e2e34fa74e_1.0
7b8aa2b9-560f-34a6-aa4b-153d1abfb542_1.0
ea3b717c-d66b-30dc-8b37-964d9688295f_1.0
d8aa6383-0da4-3c39-adba-c324bb24a494_2.0
8ec4c4f6-8040-3142-ae99-227063559eb8_1.0
08c52cb4-31b6-3c51-a886-f75e527fe049_1.0
a4f39010-e771-3375-abce-4fa0bdb01139_1.0
73eb2c88-a7c0-36db-872f-b2fc0c1c956d_1.0
c94b0026-46e5-3183-9e02-f0d8616b020e_1.0
e2b91cd9-7941-319c-a3da-5f682c9d06eb_1.0
be62d53d-6531-38ca-a396-da52bb4b1c15_3.0
af720f25-ad8e-3226-9ab8-10186b84218a_301.0
f759d1d8-0a9b-328e-9fa9-ffa5214221ae_8.0
e100d196-5e82-32e4-80e9-ac45c07a498c_1.0
cf5cbdbd-1496-33e7-b024-21ffcc72c354_1.0
c9c4584c-0a47-3132-9a36-3494f713fadc_1.0
16779668-a3e3-34ef-b17f-46a64c6338e8_1.0
73eba694-db49-372f-8d0a-860018561e36_1.0


In [ ]:
# delete any row in big_df2 whose 'course_and_section' value matches an item
# on the duplicated_course_and_section_list
# save to new df

big_df3 = big_df2[~big_df2['course_and_section'].isin(duplicated_course_and_section_list)]

# check that big_df3 has number of rows equal to len(big_df2) minus num_duplicated_rows
print(f"length of big_df2: {len(big_df2)}")
print(f"number of duplicated rows in big_df2: {num_duplicated_rows}")
print(f"big_df3 should have {len(big_df2) - num_duplicated_rows} rows")
print(f"length of big_df3: {len(big_df3)}")

length of big_df2: 365133
number of duplicated rows in big_df2: 136031
big_df3 should have 229102 rows
length of big_df3: 229102


In [ ]:
# do the same to grades_df2: drop all rows that are in duplicated_course_and_section_list

grades_df3 = grades_df2[~grades_df2['course_and_section'].isin(duplicated_course_and_section_list)]

# check old and new lengths of grades_dfs
print(f"length of grades_df2: {len(grades_df2)}")
print(f"length of grades_df2 - length of duplicated_course_and_section_list: {len(grades_df2) - len(duplicated_course_and_section_list)}")
print(f"length of grades_df3: {len(grades_df3)}")

# it's okay that the length of grades_df3 is not exactly equal to the length of grades_df2
# minus the length of the duplicated course and section list because although we assume
# that each course in the duplicated course and section list has one correlate row in
# grades_df2, that may not be the case

length of grades_df2: 193262
length of grades_df2 - length of duplicated_course_and_section_list: 144469
length of grades_df3: 165432


In [ ]:
# check now that the 'course_and_section' column uniquely identifies both big_df3 and grades_df3
print(f"rows in big_df3: {len(big_df3)}")
print(f"unique row values in 'course_and_section' column in big_df3: {big_df3['course_and_section'].nunique()}")
print()
print(f"rows in grades_df3: {len(grades_df3)}")
print(f"unique row values in 'course_and_section' column in grades_df3: {grades_df3['course_and_section'].nunique()}")


rows in big_df3: 229102
unique row values in 'course_and_section' column in big_df3: 229102

rows in grades_df3: 165432
unique row values in 'course_and_section' column in grades_df3: 165432


In [ ]:
# now can merge big_df3 and grades_df3 on 'course_and_section' column
# we want to do an inner join because we only want to keep rows that have a match in the 'course_and_section' column
# if big_df3 has info in a row about a section that doesn't have a row with its grades in grades_df3,
# we don't want that row (and vice versa)

# make new df

big_df4 = pd.merge(big_df3, 
                   grades_df3, 
                   how = 'inner',
                   on = 'course_and_section')

In [ ]:
# check
big_df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162710 entries, 0 to 162709
Data columns (total 43 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   section_uuid          162710 non-null  object 
 1   course_offering_uuid  162710 non-null  object 
 2   section_type          162710 non-null  object 
 3   section_number        162710 non-null  float64
 4   room_uuid             66778 non-null   object 
 5   schedule_uuid         162710 non-null  object 
 6   instructor_id         162682 non-null  float64
 7   facility_code         66778 non-null   object 
 8   room_code             64871 non-null   object 
 9   start_time            162710 non-null  float64
 10  end_time              162710 non-null  float64
 11  mon                   162710 non-null  object 
 12  tues                  162710 non-null  object 
 13  wed                   162710 non-null  object 
 14  thurs                 162710 non-null  object 
 15  

In [ ]:
big_df4.sample(10)

,section_uuid,course_offering_uuid,section_type,section_number,room_uuid,schedule_uuid,instructor_id,facility_code,room_code,start_time,...,f_count,s_count,u_count,cr_count,n_count,p_count,i_count,nw_count,nr_count,other_count
36950,619d3f5a-9965-3d54-b9da-886f4c32eaa7,2958d93a-2167-346a-8cea-dde6bd5b6909,fld,3.0,NaN,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,473859.0,NaN,NaN,-1.0,...,0,0,0,0,0,0,0,0,0,0
70909,e497314d-2a30-3917-9a3f-3542aaa33162,14902e49-3e34-3e08-9301-c132ee1d91f4,ind,25.0,NaN,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,988245.0,NaN,NaN,-1.0,...,0,7,0,0,0,8,0,0,0,0
136486,c3e1c71e-679d-3420-9cde-00398fb42977,e25021bc-79a8-3606-a41e-e3e381cfa7f9,lec,2.0,7d4dd9b8-4a5a-3d7b-b66b-525583c0cb4f,acfa659f-afaa-355c-a3a9-16d23ad183b9,2600269.0,0140,3335,780.0,...,0,0,0,0,0,0,0,0,0,0
20697,54d20dfc-5b41-3f64-ad8d-bfdb8e112a16,e482560e-1552-3827-b94d-4a9a1222fb6b,ind,8.0,NaN,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,4991968.0,NaN,NaN,-1.0,...,0,0,0,0,0,0,0,0,0,0
17564,846b6707-5f11-3785-b9f5-115c08b9d579,6ed95cb7-15ef-3dbf-98aa-46c75825623d,ind,40.0,NaN,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,2600403.0,NaN,NaN,-1.0,...,0,0,0,0,0,0,0,0,0,0
108930,d4b83e43-fd93-380c-b614-f34b99b72bce,37dc1ffb-f94d-3db3-bbac-0b3ee4f44fd6,lec,1.0,2f5347d3-1465-390a-9600-3a6bfe2497e5,3f78e049-8448-355b-b5a0-725b0f85f1d1,986112.0,0085,2235,480.0,...,0,0,0,0,0,0,0,0,0,0
155362,07c4a9e0-f5ec-3ab4-953a-9197200010f4,38a98d18-5d38-3385-8b2b-20e97fa370b2,fld,43.0,NaN,739ab54a-17e8-3428-b64a-5b60640a35ff,2601476.0,NaN,NaN,360.0,...,0,0,0,0,0,0,0,0,0,0
117363,9adc7744-5647-367b-8fa2-af86f857d103,d4d989f8-0fa9-3e9a-9580-1457e28deb3a,lec,1.0,ce4a2d03-9575-3700-9fce-eb7b64af6c82,b19d06a0-0c85-3e15-91e5-bd30a21074ea,4195808.0,0408,2535,780.0,...,0,0,0,0,0,0,0,0,0,0
55273,1f3d6ef8-d5cc-36e7-ac10-24fd06133a5d,c94e7737-3563-36fa-9f66-cfc9ed7a1163,ind,19.0,NaN,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,2600316.0,NaN,NaN,-1.0,...,0,0,0,0,0,0,0,0,0,0
94939,0028ce75-049b-31f7-a4d3-554f56ed60c6,96bc2ba0-d66b-385d-8f6e-79a3d86fed68,fld,1.0,NaN,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,940942.0,NaN,NaN,-1.0,...,0,0,0,0,0,0,0,0,0,0


Now the big csv including all the data in all csv files given in this dataset in Kaggle is ready to be further cleaned and used for machine learning.

# Export to csv

In [ ]:
big_df4.to_csv('/content/drive/MyDrive/Coding Dojo/Project 2/all_grades_data.csv')